# 5. 그룹 연산

복잡한 데이터를 어떤 기준에 따라 여러 그룹으로 나눠서 관찰하는 것도 좋은 방법이다.<br>
이처럼 특정 기준을 적용하여 몇 개의 그룹으로 분할하여 처리하는 것을 그룹 연산이라고 부른다.
<br><br>
특히 그룹 연산은 데이터를 집계, 변환, 필터링하는데 효율적이다.<br>
3단계의 과정으로 이루어지고, 데이터를 그룹으로 분할하는 1단계 과정은 판다스 `groupby()` 메소드를 사용한다.
<br><br>
> 1단계) 분할(split): 데이터를 특정 조건에 의해 분할<br>
> 2단계) 적용(apply): 데이터를 집계, 변환, 필터링하는데 필요한 메소드 적용<br>
> 3단계) 결합(combine): 2단계의 처리 결과를 하나로 결합<br>

## 5-1. 그룹 객체 만들기(분할 단계)

- 1개 열을 기준으로 그룹화
`groupby()` 메소드는 데이터프레임의 특정 열을 기준으로 데이터프레임을 분할하여 그룹 객체를 반환한다.<br>
기준이 되는 열은 1개도 가능하고, 여러 열을 리스트로 입력할 수도 있다.

- 그룹 연산(분할): `DataFrame 객체.groupby(기준이 되는 열)`

먼저 열 1개를 기준으로 분할하여 그룹 객체를 만드는 방법이다.<br>
'titanic' 데이터셋에서 5개의 열을 선택하여 만든 데이터프레임을 'class' 열을 기준(키)으로 하여 그룹으로 나눈다.


In [12]:
import pandas as pd
import seaborn as sns

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

print('승객 수:', len(df))
print(df.head())
print('\n')

# class 열을 기준으로 분할
grouped = df.groupby(['class'])
print(grouped)


승객 수: 891
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0




---

'class' 열에는 'First', 'Second', 'Third'라는 3개의 값이 들어 있다.<br>
이 3개의 값을 기준으로 891명의 승객 데이터가 3개의 그룹으로 나누어진다.<br>
반복문은 이용하여 그룹 객체의 내용을 출력하면
'First' 클래스에 216명, 'Second' 클래스에 184명, 'Third' 클래스에 491명의 승객 데이터가 있다.<br>
또한 각 승객의 행 인덱스가 그대로 유지되는 것을 알 수 있다.


In [13]:
# 그룹 객체를 iteration으로 출력: head() 메소드로 첫 5행만을 출력
for key, group in grouped:
    print('* key :', key)
    print('* number :', len(group))
    print(group.head())
    print('\n')

* key : First
* number : 216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1


* key : Second
* number : 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


* key : Third
* number : 491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




---

앞에서 생성한 그룹 객체(grouped)에 연산 메소드를 적용할 수 있다.<br>
다음의 예제에서 grouped 객체에 들어 있는 3개 그룹에 대하여 그룹별 평균값을 구하기 위해 `mean()` 메소드를 적용한다.<br>
이때 연산이 가능한 열에 대해서만 선택적으로 연산을 수행한다.<br>
따라서 문자열 데이터를 갖는 'sex', 'class' 열은 제외하고, 숫자형 데이터를 가진 열에 대해서는 평균값을 계산한다.<br>
1등석의 평균 나이가 38세로 가장 많고, 구조 확률도 63% 정도로 가장 높다는 정보를 얻을 수 있다.


In [14]:
# 연산 메소드 적용
average = grouped.mean()
print(average)

              age       fare  survived
class                                 
First   38.233441  84.154687  0.629630
Second  29.877630  20.662183  0.472826
Third   25.140620  13.675550  0.242363


---

그룹 객체에 `get_group()` 메소드를 적용하면 특정 그룹만을 선택할 수 있다.<br>
grouped 객체의 3개 그룹 중 키 값이 'Third'인 3등석 승객 데이터를 가진 그룹을 따로 선택하여 추출한다.

In [15]:
# 개별 그룹 선택하기
group3 = grouped.get_group('Third')
print(group3.head())

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0


---

- 여러 열을 기준으로 그룹화

이번에는 `groupby()` 메소드에 여러 개의 열을 리스트로 전달하는 방법을 알아보자.<br>
여러 개의 기준 값을 사용하기 때문에 반환되는 그룹 객체의 인덱스는 다중 구조를 갖는다.<br>

- 그룹 연산(분할): `DataFrame 객체.groupby(기준이 되는 열의 리스트)`

`groupby()` 메소드에 두 열('class', 'sex')을 인자로 전달하면 두 열이 갖는 원소 값들로 만들 수 있는 모든 조합으로 키를 생성한다.<br>
그리고 조합된 키를 기준으로 그룹 객체를 만든다.<br>
'class' 열에는 'First', 'Second', 'Third'라는 3개의 값이 들어 있고, 'sex' 열에는 'male', 'female' 2개의 값이 들어 있다.<br>
가능한 조합은 ('class', 'sex') 형식의 튜플로 지정되는데, ('First', 'female')을 포함하여 모두 6개의 키가 조합으로 만들어진다.<br>
모두 6개의 그룹으로 상세하게 구분할 수 있다.


In [16]:
# class 열, sex 열을 기준으로 분할
grouped_two = df.groupby(['class', 'sex'])

# grouped_two 그룹 객체를 iteration으로 출력
for key, group in grouped_two:
    print('* key :', key)
    print('* number :', len(group))
    print(group.head())
    print('\n')


* key : ('First', 'female')
* number : 94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1


* key : ('First', 'male')
* number : 122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0


* key : ('Second', 'female')
* number : 76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1


* key : ('Second', 'male')
* number : 108
     age   sex   class  fare  survived
17   Na

---

grouped_two 객체의 각 그룹에 `mean()` 메소드를 적용한다.<br>
이때 데이터프레임이 반환되는데, 키가 되는 2개의 열('class', 'sex')의 값으로부터 2중 멀티 인덱스가 저장된다.<br><br>

실행 결과를 보면 여성의 생존률이 남성에 비해 월등히 높다.<br>
다만, 3등석 여성 승객의 생존율은 50%에 불과하여, 1~2등석 여성 승객의 90%대 대비 상대적으로 낮다는 사실을 알 수 있다.


In [17]:
# grouped_two 그룹 전체의 연산 메소드 적용
average_two = grouped_two.mean()
print(average_two)
print('\n')
print(type(average_two))



                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


<class 'pandas.core.frame.DataFrame'>
